### Detecção de Patologias Oculares em Imagens Reflexo Vermelho Utilizando Descritores de Cor

##### Carregar bibliotecas necessárias

In [1]:
import glob
import cv2

import numpy as np

# dominant color
from sklearn.cluster import KMeans

# color moments
from scipy.stats import skew, kurtosis

import pandas as pd

# classifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

### 1. Carregar Imagens

In [2]:
DIR_IMGS_NORMAIS  = "./data/olhos/normal"
DIR_IMGS_PROBLEMA = "./data/olhos/problema"

DIR_MARCACOES_NORMAIS  = "./data/marcacoes/normal"
DIR_MARCACOES_PROBLEMA = "./data/marcacoes/normal"

In [3]:
def load_images(path, label):
    imgs, labels = [], []
    
    # select every .jpg file from directory
    for file in glob.iglob(path + '/*.jpg'):
    
        img  = cv2.imread(file)
        img  = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                   
        imgs.append(img)
        labels.append(label)
    
    return imgs, labels

### 2. Realizar Pré Processamento

##### Equalização de Histograma

In [4]:
def equalize_hist(img):
    
    r, g, b = cv2.split(img)
    
    r_eq = cv2.equalizeHist(r)
    g_eq = cv2.equalizeHist(g)
    b_eq = cv2.equalizeHist(b)
        
    img_eq = cv2.merge((r_eq, g_eq, b_eq))
    
    return img_eq

##### Contraste CLAHE

In [5]:
def apply_clahe(img):
    # clahe creation (todo: optimize!)
    clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(8, 8))
    
    img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    
    l, a, b = cv2.split(img_lab)
    l = clahe.apply(l)

    img_clahe = cv2.merge((l, a, b))
    img_clahe = cv2.cvtColor(img_clahe, cv2.COLOR_LAB2RGB)
        
    return img_clahe

##### Cores Opostas

In [6]:
def opposite_colors(img):
    return 255 - img

##### Função Geral

In [7]:
def apply_preprocess(img, method):
    if method == 'eqlze':
        return equalize_hist(img)
    
    elif method == 'clahe':
        return apply_clahe(img)
    
    elif method == 'oppcl':
        return opposite_colors(img)
    
    else:
        return img

### 3. Definir Espectro de Cor

##### Função Geral

In [8]:
def define_spectrum(img, spectrum):
    if spectrum == 'hsv':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    elif spectrum == 'lab':
        return cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    
    else:
        return img

### 4. Aplicar Máscara (ou não)

In [9]:
def apply_masks(imgs, masks_path):
    
    imgs_mask = []
    
    for img, file in zip(imgs,
                         glob.iglob(masks_path + '/*.jpg')):
        
        mask     = cv2.imread(file, 0)
        img_mask = cv2.bitwise_and(img, img, mask = mask)
        imgs_mask.append(img_mask)
    
    return imgs_mask

### 5. Extrair características com descritores de cor

##### Histograma de Cor

In [10]:
def color_histogram(img):
    hist = []
    
    for channel in cv2.split(img):
        hist.extend(cv2.calcHist([channel], [0], None, [256], [0, 256]))
    
    hist = np.concatenate(np.array(hist), axis = 0)
    
    return hist

##### Descritor de Cor Dominante

In [11]:
def dominant_color(img):
    
    img = img.reshape((img.shape[0] * img.shape[1],3))
    clt = KMeans(n_clusters=3)
    clt.fit(img)

    dominant = np.concatenate(clt.cluster_centers_, axis = 0)
    
    return dominant

##### Momentos de Cor

In [12]:
def color_moments(img):
    
    moments = []
    
    for ch in cv2.split(img):
        
        # first color moment
        mean      = np.mean(ch)
        moments.append(mean)
        
        # second color moment
        variance  = np.var(ch)
        moments.append(variance)

        # third color moment
        skewness  = skew(ch.reshape(-1))
        moments.append(skewness)
        
        # forth color moment
        kurt      = kurtosis(ch.reshape(-1))
        moments.append(kurt)
    
    moments = np.array(moments)
    
    return moments

##### Função Geral

In [13]:
def apply_color_descriptor(img, descriptor):
    if descriptor == 'histogrm':
        return color_histogram(img)
    
    elif descriptor == 'dominant':
        return dominant_color(img)
    
    else:
        return color_moments(img)

### 6. Classificação e Funções Auxiliares

##### Função para salvar os resultados do modelo em um .csv

In [14]:
def save_results(chosen_params, cv_scores, scores):
    # chosen_params
    result = f'{chosen_params};'
    
    # save all scores in result
    for score in scores:
        result += f'{score};'
    
    # cross validation mean
    result += f'{np.mean(cv_scores)};'
    
    # cross validations
    for score in cv_scores:
        result += f'{score};'
    
    # break line
    result += '\n'
    
    # save results
    with open('results.csv','a') as file:
        file.write(result)
    
    # return accuracy
    return scores[7]

##### Funções auxiliares para calcular scores do modelo

In [15]:
def get_metrics(tn, fp, fn, tp):
    # precision
    if tp + fp > 0:
        pre = tp / (tp + fp)
    else:
        pre = 0
    
    # sensibility
    sen = tp / (tp + fn)
    
    # especificity
    esp = tn / (tn + fp)
    
    # acuracy
    acc = (tp + tn) / (tp + tn + fp + fn)
    
    return pre, sen, esp, acc

In [16]:
def get_model_scores(model, X_test, y_test):
    # predict
    y_pred             = model.predict(X_test)
    
    # get confusion matrix
    tn, fp, fn, tp     = (sklearn.metrics.confusion_matrix(y_test, y_pred)).ravel()
    
    # get_scores
    pre, sen, esp, acc = get_metrics(tn, fp, fn, tp)
    
    results = [tn, fp, fn, tp, pre, sen, esp, acc]
    
    return results

##### SVM

In [17]:
def train_with_SVM(features, labels):
    
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.33, random_state = 42)
    
    # model creation and parameters optimization
    parameters = [{'kernel': ['linear', 'rbf'],
                   'C': [1, 10, 100, 1000],
                   'gamma': [0.01, 0.001, 0.0001]}]
    
    clf        = SVC()
    model      = GridSearchCV(clf, parameters, scoring = 'accuracy').fit(X_train, y_train)
    
    # cross-validation scores
    #cv_scores = cross_val_score(model, features, labels, cv = 10)
    cv_scores  = range(10)
    
    # model scores
    scores    = get_model_scores(model, X_test, y_test)

    return cv_scores, scores 

##### Random Forest

In [18]:
def train_with_RF(features, labels):
    
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.33, random_state = 42)

    # model creation and parameters optimization
    parameters = [{'criterion': ['gini', 'entropy'],
                   'max_features': ['auto', 'sqrt', 'log2'],
                   'class_weight': ['balanced', 'balanced_subsample']}]
    
    clf        = RandomForestClassifier()
    model      = GridSearchCV(clf, parameters, scoring = 'accuracy').fit(X_train, y_train)
    
    # cross-validation scores
    #cv_scores = cross_val_score(model, features, labels, cv = 10)
    cv_scores  = range(10)
    
    # model scores
    scores    = get_model_scores(model, X_test, y_test)

    return cv_scores, scores

##### Função Geral

In [19]:
def define_model(features, labels, classifier):
    
    if classifier == 'svm':
        return train_with_SVM(features, labels)
    
    else:
        return train_with_RF(features, labels)

### #7 HyperOpt!

##### Função para checar se os métodos escolhidos já foram escolhidos anteriormente

In [20]:
def not_chosen(chosen_params):
    
    data    = (pd.read_csv('results.csv', sep = ';')).reset_index()
    methods = data.iloc[:, 0].values
    
    for method in methods:
        if method == chosen_params:
            return False
    
    return True

##### Espaço de Busca

In [21]:
search_space ={
    # pre-processing: histogram equalization, clahe contrast,
    #                 opposite colors, no pre-processing
    'pre'    : hp.choice('pre', ['eqlze', 'clahe', 'oppcl', 'nopre']),
    
    # spectrum: rgb, hsv, lab
    'esp'    : hp.choice('esp', ['rgb','hsv','Lab']),
    
    # mask: apply mask or not
    'mask'   : hp.choice('mask', ['appmask', 'nopmask']),
    
    # color-descriptors: color histogram, dominant colors, color moments
    'desc'   : hp.choice('desc', ['histogrm', 'dominant', 'imoments']),
    
    # classifier: svm or random forest
    'class'  : hp.choice('class', ['svm', 'rfr'])
}

##### Função otimizadora

In [22]:
def optimize(params):
    chosen_params = f"{params['pre']}_{params['esp']}_{params['mask']}_{params['desc']}_{params['class']}"
    print(chosen_params)
    
    # if the chosen params were not previously selected...
    if(not_chosen(chosen_params)):
        # load images
        n_imgs, n_labels = load_images(DIR_IMGS_NORMAIS, 0)
        p_imgs, p_labels = load_images(DIR_IMGS_PROBLEMA, 1)

        # pre-processing
        n_imgs = [apply_preprocess(img, params['pre']) for img in n_imgs]
        p_imgs = [apply_preprocess(img, params['pre']) for img in p_imgs]

        # transform spectrum
        n_imgs = [define_spectrum(img, params['esp']) for img in n_imgs]
        p_imgs = [define_spectrum(img, params['esp']) for img in p_imgs]

        # apply mask (or not)
        if params['mask'] == 'appmask':
            n_imgs = apply_masks(n_imgs, DIR_MARCACOES_NORMAIS)
            p_imgs = apply_masks(p_imgs, DIR_MARCACOES_PROBLEMA)

        # concat vectors
        imgs   = n_imgs + p_imgs
        labels = n_labels + p_labels

        # full features vector
        features = []

        # extract features
        for img in imgs:
            feats = apply_color_descriptor(img, params['desc'])
            features.append(feats)

        # model creation and score calculation : take some time...
        cv_scores, scores = define_model(features, labels, params['class'])

        # save scores
        acc               = save_results(chosen_params, cv_scores, scores)
        return 1 - acc
    
    return 1 - 0.74

##### MAIN!

In [ ]:
def main():
    while True:
        best = fmin(optimize, search_space, algo = tpe.suggest, max_evals = 50)
        
main()

clahe_Lab_appmask_dominant_rfr                                                                                         
clahe_hsv_nopmask_imoments_svm                                                                                         
  2%|▉                                            | 1/50 [01:59<1:37:53, 119.86s/trial, best loss: 0.20512820512820518]